In [1]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malay-function-call/resolve/main/function-calls.jsonl
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malay-function-call/resolve/main/function-calls-complex.jsonl

In [2]:
from malaysian_sft import accept as accept_fn

In [3]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [4]:
import json
import random
from glob import glob
from tqdm import tqdm

In [13]:
data = []

with open('function-calls.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c])
                except:
                    failed = True
                    continue


            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
            
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
            
        if isinstance(l['function_call'], dict):
            function = json.dumps(l['function_call'])
        else:
            function = l['function_call']

        data.append({
            'prompt_input': function,
            'input': '\n'.join(texts),
            'output': None,
        })
            
len(data)

179450it [00:20, 8833.44it/s] 


179339

In [14]:
data_complex = []

with open('function-calls-complex.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c])
                except:
                    failed = True
                    continue


            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue
        
        if isinstance(l['function_call'], dict):
            function = json.dumps(l['function_call'])
        else:
            function = l['function_call']

        data_complex.append({
            'prompt_input': function,
            'input': '\n'.join(texts),
            'output': None,
        })
            
len(data_complex)

24984

In [32]:
filtered = []
for d in data + data_complex:
    if not isinstance(d['prompt_input'], str):
        continue
    if d['prompt_input'] is None:
        continue
    
    if d['input'] is None:
        continue
        
    if not isinstance(d['input'], str):
        continue
        
    if d['output'] is not None:
        continue
        
    filtered.append(d)
    
len(filtered)

204323

In [26]:
len(data_complex) + len(data)

204323

In [29]:
import pandas as pd

df = pd.DataFrame(filtered)
df.head()

,prompt_input,input,output
0,"{\n ""name"": ""get_news_headlines"",\n ""des...",<manusia>: Bolehkah anda memberitahu saya taju...,None
1,"{\n ""name"": ""dapatkan_data_pasaran"",\n ""...",<manusia>: Bolehkah anda memberitahu saya data...,None
2,"{\n ""name"": ""read_product_reviews"",\n ""d...",<manusia>: Saya berminat untuk membaca ulasan ...,None
3,"{\n ""name"": ""generate_password"",\n ""desc...",<manusia>: I would like it to be 12 characters...,None
4,"{\n ""name"": ""check_stock_availability"",\n ...",<manusia>: I'd like to check the availability ...,None


In [16]:
df.to_parquet('prepared-function-call.parquet')

In [36]:
print(filtered[4]['prompt_input'])

{
    "name": "check_stock_availability",
    "description": "Check the availability of a product in stock",
    "parameters": {
        "type": "object",
        "properties": {
            "product_id": {
                "type": "string",
                "description": "The ID of the product to check"
            },
            "quantity": {
                "type": "integer",
                "description": "The desired quantity of the product"
            }
        },
        "required": [
            "product_id",
            "quantity"
        ]
    }
}

{
    "name": "place_order",
    "description": "Place an order for a product",
    "parameters": {
        "type": "object",
        "properties": {
            "product_id": {
                "type": "string",
                "description": "The ID of the product to order"
            },
            "quantity": {
                "type": "integer",
                "description": "The quantity of the product to order"
            }

In [22]:
data[0]['input']

'<manusia>: Bolehkah anda memberitahu saya tajuk berita terkini untuk Amerika Syarikat?\n<bot>: {"name": "get_news_headlines", "arguments": {"country": "Amerika Syarikat"}}\n<manusia>: Itu menarik. Bagaimana pula dengan berita di Perancis?\n<bot>: {"name": "get_news_headlines", "arguments": {"country": "Perancis"}}'

In [37]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-function-call.parquet",
    path_in_repo='data/function_call-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/baddecb92d3203386288983feb3b3a1495686c9d', commit_message='Upload data/function_call-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='baddecb92d3203386288983feb3b3a1495686c9d', pr_url=None, pr_revision=None, pr_num=None)